<a href="https://colab.research.google.com/github/EliasVillalvazo/master_deg_documents/blob/main/pytorch_tutorial/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution Neural Networks

Made of neurons as well
They work on image data and work on images. Their purpose is to learn some features from the images and then perform the classification task

# Max Pooling
Downsample an images by applying a max filter to subregions of the image. The maximum value is kept

Reduces the computational cost by reducing the size (parameters to learn) and helps to avoid overfitting.


# Size after a convolution:

(W - F + 2P) / S + 1
W = Width
F = Filter size
P = Padding
S = Stride

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [22]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyperparameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

# define transformations

transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [23]:
# Load datasets
train_dataset = torchvision.datasets.CIFAR10(root='.', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='.', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [24]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [28]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, stride=2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc = nn.Linear(16*5*5, 120) # 16 channels multiplied by the width and length of the image after convolutions and pooling. Apply the formula
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = ConvNet().to(device)

In [29]:
# create loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

In [30]:
# Do training loop now

# define number of total steps
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  # loop over the batches
  for i, (images, labels) in enumerate(train_loader):
    # reshape images first and send to device
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    # backprop
    loss.backward()
    # update parameters
    optimizer.step()

    if (i + 1) % 100 == 0:
      print(f'current epoch: {epoch+1} / {num_epochs}; current step: {i+1} / {n_total_steps}; loss = {loss.item():.4f}')

# test step
with torch.no_grad():
  n_correct_predictions = 0
  n_samples = 0
  for images, labels in test_loader:
    # reshape images first and send to device
    images = images.to(device)
    labels = labels.to(device)

    # calculate the predictions
    output_t = model(images)
    _, predictions = torch.max(output_t, 1) # returns value and index
    n_samples += labels.shape[0]
    n_correct_predictions += (predictions == labels).sum().item()

  acc = 100.0 * n_correct_predictions / n_samples
  print(acc)

current epoch: 1 / 4; current step: 100 / 12500; loss = 2.3015
current epoch: 1 / 4; current step: 200 / 12500; loss = 2.3656
current epoch: 1 / 4; current step: 300 / 12500; loss = 2.2971
current epoch: 1 / 4; current step: 400 / 12500; loss = 2.3079
current epoch: 1 / 4; current step: 500 / 12500; loss = 2.2690
current epoch: 1 / 4; current step: 600 / 12500; loss = 2.3110
current epoch: 1 / 4; current step: 700 / 12500; loss = 2.3249
current epoch: 1 / 4; current step: 800 / 12500; loss = 2.3208
current epoch: 1 / 4; current step: 900 / 12500; loss = 2.3130
current epoch: 1 / 4; current step: 1000 / 12500; loss = 2.2927
current epoch: 1 / 4; current step: 1100 / 12500; loss = 2.2707
current epoch: 1 / 4; current step: 1200 / 12500; loss = 2.3389
current epoch: 1 / 4; current step: 1300 / 12500; loss = 2.2820
current epoch: 1 / 4; current step: 1400 / 12500; loss = 2.3070
current epoch: 1 / 4; current step: 1500 / 12500; loss = 2.3200
current epoch: 1 / 4; current step: 1600 / 12500;